## Instalando e Carregando os Pacotes

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [ ]:
# Instala o pacote iPython-sql 
# https://pypi.org/project/ipython-sql/
!pip install ipython-sql

In [ ]:
# Imports
import pandas as pd
import sqlite3

## Banco Relacional, Python e SQL Para Análise de Dados

In [ ]:
# Cria um dataframe
dados = pd.DataFrame({'nome': ['Siri', 'Alexa', 'Cortana'],
                      'idade': [28, 47, 18],
                      'cargo': ['Analista de Dados', 'Cientista de Dados', 'Engenheiro de Dados']})

In [ ]:
dados.head()

In [ ]:
# Apaga o banco de dados
#!del 'database/dbprojeto1.db'
!rm 'database/dbprojeto1.db'

In [ ]:
# Criamos a conexão e um banco de dados com nome dbprojeto1
cnn = sqlite3.connect('database/dbprojeto1.db')

In [ ]:
# Copia o dataframe para dentro do banco de dados como uma tabela
dados.to_sql('funcionarios', cnn)

In [ ]:
# Carregamos a extensão SQL
%load_ext sql

In [ ]:
# Definimos o banco de dados
%sql sqlite:///database/dbprojeto1.db

> Agora executamos nossas consultas SQL usando diretamente Linguagem SQL dentro do Jupyter Notebook.

In [ ]:
%%sql

SELECT * FROM funcionarios

In [ ]:
%%sql

SELECT count(*) FROM funcionarios

In [ ]:
%%sql

SELECT avg(idade) as 'idade_media' FROM funcionarios

## Banco Relacional, Python e SQL Para Análise de Dados

Temos em mãos um arquivo com dados de pacientes que desenvolveram ou não diabetes. Precisamos gerar uma amostra de dados com os pacientes com mais de 50 anos e para cada um deles indicar em uma nova coluna se o paciente está normal (BMI menor que 30) ou obeso (BMI maior ou igual a 30). Então devemos gerar um novo arquivo CSV e encaminhar para o Cientista de Dados.

In [ ]:
df = pd.read_csv('dataset/diabetes.csv')

In [ ]:
type(df)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# Copia o dataframe para dentro do banco de dados como uma tabela
df.to_sql('diabetes', cnn)

In [ ]:
%%sql

SELECT count(*) FROM diabetes

In [ ]:
%%sql

SELECT Age, Glucose, Outcome FROM diabetes WHERE Glucose > 190

In [ ]:
df.columns

In [ ]:
%%sql

CREATE TABLE pacientes (Pregnancies INT,
                        Glucose INT,
                        BloodPressure INT,
                        SkinThickness INT,
                        Insulin INT,
                        BMI DECIMAL(8, 2),
                        DiabetesPedigreeFunction DECIMAL(8, 2),
                        Age INT,
                        Outcome INT);

In [ ]:
%%sql

SELECT * FROM pacientes

### movendo de uma tabela para outra tudo aquilo em que a idade do paciente é maior que 50.

In [ ]:
%%sql

INSERT INTO pacientes(Pregnancies, 
                      Glucose, 
                      BloodPressure, 
                      SkinThickness, 
                      Insulin, 
                      BMI, 
                      DiabetesPedigreeFunction, 
                      Age, 
                      Outcome) 
SELECT Pregnancies, 
       Glucose, 
       BloodPressure, 
       SkinThickness, 
       Insulin, 
       BMI, 
       DiabetesPedigreeFunction, 
       Age, 
       Outcome 
FROM diabetes WHERE Age > 50;

In [ ]:
%%sql

SELECT * FROM pacientes

In [ ]:
%%sql 

ALTER TABLE pacientes
ADD Perfil VARCHAR(10);

In [ ]:
%%sql

SELECT * FROM pacientes

In [ ]:
%%sql

UPDATE pacientes
SET Perfil = 'Normal'
WHERE BMI < 30;

In [ ]:
%%sql

SELECT * FROM pacientes

In [ ]:
%%sql

UPDATE pacientes
SET Perfil = 'Obeso'
WHERE BMI >= 30;

In [ ]:
%%sql

SELECT * FROM pacientes

## Carregando os Dados no Pandas e Salvando o CSV

In [ ]:
# Query para consultar tabela de pacientes.
query = cnn.execute("SELECT * FROM pacientes")
query

In [ ]:
# List Comprehension
cols = [coluna[0] for coluna in query.description]
cols

In [ ]:
# Gera o dataframe
resultado = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

In [ ]:
# Shape
resultado.shape

In [ ]:
# Visualiza
resultado.head()

In [ ]:
# Salva em CSV
resultado.to_csv('dataset/pacientes.csv', index = False)

# Fim